In [2]:
import numpy as np
import pandas as pd

In [3]:
# Use this because I couldn't see enough output
pd.set_option('display.max_rows', None)

In [4]:
df = pd.read_csv('export_format4.csv')

In [5]:
df.head()

,employer,period_begin,period_end,post_date,salary,sheltered,unsheltered,total
0,Orleans,1/1/2002,1/31/2002,1/29/2002,262.62,0.0,18.38,18.38
1,Orleans,2/1/2002,2/28/2002,2/22/2002,525.24,0.0,36.76,36.76
2,Orleans,3/1/2002,3/31/2002,3/27/2002,525.24,0.0,36.76,36.76
3,Orleans,4/1/2002,4/30/2002,4/26/2002,525.24,0.0,36.76,36.76
4,Orleans,5/1/2002,5/31/2002,6/11/2002,"1,021.93",0.0,71.53,71.53


In [6]:
# Ran into a problem converting salary - reading export_format.csv worked fine, 
# but reading export_format4.csv did not!
# here i notice that salary is an object, so that means string right?
# how come the export for salary was float64 for exportformat, but not exportformat4???? weird
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   employer      582 non-null    object 
 1   period_begin  582 non-null    object 
 2   period_end    582 non-null    object 
 3   post_date     582 non-null    object 
 4   salary        582 non-null    object 
 5   sheltered     582 non-null    float64
 6   unsheltered   582 non-null    float64
 7   total         582 non-null    float64
dtypes: float64(3), object(5)
memory usage: 36.5+ KB


In [8]:
# why is this not working? the comma? isn't index 4 of "1,021.93" the 1? skip the comma, it is the '.'
df['salary'] = pd.to_numeric(df['salary'])

ValueError: Unable to parse string "1,021.93" at position 4

In [11]:
# this is executed on export_format4
df = df.groupby('period_end').sum()
    

In [10]:
# this is executed on file export_format4.csv
# this is what made me go back up to the top and add cells to discover 'salary' was object not float64
df.head()

,sheltered,unsheltered,total
period_end,,,
1/31/2002,0.00,36.75,36.75
1/31/2003,0.00,89.20,89.20
1/31/2004,0.00,102.12,102.12
1/31/2005,113.80,0.00,113.80
1/31/2006,106.44,0.00,106.44


In [12]:
# So, at first I ran all these on 'export_format.csv'
# this cell was ran on export_format.csv, and it 'worked' (but the output did not match my expected result)
# when I applied the above on export_format.csv, i noticed my df did not sort appropriately
# before, the df was sorting by months instead of true linear dates
df = df.sort_values(by='period_end', key=pd.to_datetime)

In [13]:
# remember how to call help
help(df.sort_values)

Help on method sort_values in module pandas.core.frame:

sort_values(by: 'IndexLabel', axis: 'Axis' = 0, ascending: 'bool | list[bool] | tuple[bool, ...]' = True, inplace: 'bool' = False, kind: 'str' = 'quicksort', na_position: 'str' = 'last', ignore_index: 'bool' = False, key: 'ValueKeyFunc' = None) -> 'DataFrame | None' method of pandas.core.frame.DataFrame instance
    Sort by the values along either axis.
    
    Parameters
    ----------
            by : str or list of str
                Name or list of names to sort by.
    
                - if `axis` is 0 or `'index'` then `by` may contain index
                  levels and/or column labels.
                - if `axis` is 1 or `'columns'` then `by` may contain column
                  levels and/or index labels.
    axis : {0 or 'index', 1 or 'columns'}, default 0
         Axis to be sorted.
    ascending : bool or list of bool, default True
         Sort ascending vs. descending. Specify list for multiple sort
         order

In [14]:
# this is where i discovered that using groupby on export_format4, that the 'object' columns were dropped
# again....all of these 'export' csv's were exported from the same program, I'm confused why some convereted
# to pandas as object columns and some converted as float64 columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 247 entries, 1/31/2002 to 4/30/2022
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sheltered    247 non-null    float64
 1   unsheltered  247 non-null    float64
 2   total        247 non-null    float64
dtypes: float64(3)
memory usage: 7.7+ KB


In [15]:
#note that this did work on df = export_format but it did not work on df = export_format4 (because 'salary' column is not an int)
df['rolling_60'] = df['salary'].rolling(60).mean()

KeyError: 'salary'

In [35]:
# note that this cell output was executed on reading csv file 'export_format' NOT format4
# the rolling_60 values did NOT match my expected results (see file samp_salary). expected results also listed 'export_key'
df.tail()

,salary,sheltered,unsheltered,total,rolling_60
period_end,,,,,
9/30/2017,7623.59,533.66,0.0,533.66,3360.969000
9/30/2018,6231.18,436.18,0.0,436.18,3339.196500
9/30/2019,6386.66,447.06,0.0,447.06,3330.943833
9/30/2020,6711.89,469.84,0.0,469.84,3294.497000
9/30/2021,7056.93,493.98,0.0,493.98,3394.736333


In [36]:
# this was executed on export_format
df['salary'].info()

<class 'pandas.core.series.Series'>
Index: 179 entries, 1/31/1989 to 9/30/2021
Series name: salary
Non-Null Count  Dtype  
--------------  -----  
179 non-null    float64
dtypes: float64(1)
memory usage: 2.8+ KB


In [113]:
def salary_cap(rolling_60):
    for i in len(rolling_60):
        for num[i] in rolling_60:
            # well............actually this isn't going to work at all BECAUSE
            # i need this to happen within the .rolling() function, not AFTER
            # if called on the rolling_60 column, then i'm just implementing the cap end result, not within the year to year

SyntaxError: invalid syntax (2300947213.py, line 1)

In [46]:
#i wrote this function before i discovered pd.to_numeric
#....it didn't work, could you tell me why?

def int_convert(notnum):
    convert = []
    for num in notnum:
        convert += int(num)
    return convert

In [ ]:
#adding a comment to test git remote